# Imports

In [1]:
import pandas as pd

# Data Collection

In [2]:
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/train_X_ec.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/train_Y_ec.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/test_X_ec.csv

--2022-08-03 10:00:07--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/train_X_ec.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.219.160.38
Connecting to nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)|52.219.160.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27774 (27K) [text/csv]
Saving to: ‘train_X_ec.csv’

train_X_ec.csv      100%[===================>]  27.12K   128KB/s    in 0.2s    

2022-08-03 10:00:08 (128 KB/s) - ‘train_X_ec.csv’ saved [27774/27774]

--2022-08-03 10:00:08--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/E_Commerce/train_Y_ec.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.

In [3]:
train_X_df = pd.read_csv('train_X_ec.csv')
train_Y_df = pd.read_csv('train_Y_ec.csv', header=None).squeeze()
test_X_df = pd.read_csv('test_X_ec.csv')

# Implementation

In [4]:
train_X_df.isna().sum()

Time Spent on Website     0
Duration of Membership    0
Time Spent on App         0
Session Duration\t        0
dtype: int64

In [5]:
train_X_df.head()

,Time Spent on Website,Duration of Membership,Time Spent on App,Session Duration\t
0,69.156561,20.457356,32.219553,62.186251
1,67.975829,19.787373,30.887689,57.816544
2,66.702558,18.986008,32.711838,58.286399
3,65.371799,19.474788,29.409797,62.118480
4,66.551924,18.933956,32.462605,59.442156


In [6]:
train_Y_df.head()

0    897.056602
1    708.029955
2    725.771404
3    723.439029
4    753.824173
Name: 0, dtype: float64

In [14]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.linear_model import Ridge
pipe = Pipeline(steps = [('scaler', StandardScaler()),
                         ('regressor',Ridge())])

In [18]:
from sklearn.model_selection import GridSearchCV
parameters = [
    {
     'alpha':[1,3,5,0.1,0.01,0.001],
     'solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    }
]
grid_search = GridSearchCV(Ridge(), parameters, scoring='neg_mean_squared_error', refit=True, cv=10) 
grid_search.fit(train_X_df, train_Y_df)

GridSearchCV(cv=10, estimator=Ridge(),
             param_grid=[{'alpha': [1, 3, 5, 0.1, 0.01, 0.001],
                          'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                     'sparse_cg', 'sag', 'saga']}],
             scoring='neg_mean_squared_error')

In [19]:
print(grid_search.best_params_)

{'alpha': 1, 'solver': 'saga'}


In [21]:
grid_search.cv_results_['mean_test_score'].max()

-221.16115476306035

In [22]:
model = grid_search.best_estimator_
predicted_test_Y = model.predict(test_X_df)

# Submission 
Writing the predicted values to CSV file

In [23]:
predicted_test_Y = pd.DataFrame(predicted_test_Y).to_csv('predicted_test_Y_ec.csv', header=None, index=False)

In [24]:
from google.colab import files

#predicted_test_Y.to_csv('predicted_test_Y_ec.csv')
files.download('predicted_test_Y_ec.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>